<a href="https://colab.research.google.com/github/alifele/Brain_Computer_Interface-Competition-/blob/master/Dense_for_FreqContent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import sys
import numpy as np
import matplotlib.pyplot as plt
sys.path.append('/content/drive/My Drive/EEG_BCI/')
import datafetch
import plotters
import random

In [2]:
subj_1 = datafetch.fetch_subject(12)
subj_1.keys()

dict_keys(['test_data', 'train_data_class1', 'train_data_class2', 'train_data_class3', 'train_data_class4'])

In [0]:
def selector_desing(subject):
  selector = []
  for i in subject:
    for j in range(1,5):
      for k in range(18):
        selector.append((i,j,k))

  selector = np.array(selector)
  np.random.shuffle(selector)

  return selector

In [0]:
def selector_desing_test_data(subject):
  selector = []
  for i in subject:
    for j in range(1,5):
      for k in range(18,25):
        selector.append((i,j,k))

  selector = np.array(selector)
  np.random.shuffle(selector)
  return selector

In [0]:
global indexes
global test_data_indexes
global selector
global selector_test_data
global f_cut

f_cut=100
selector_test_data = selector_desing_test_data([1])
selector = selector_desing([1])
indexes = [i for i in range(18)]
test_data_indexes = [i for i in range(18,25)]

In [0]:
t = np.linspace(0,3,7200)
f = np.linspace(1/3, 2400, 7200)
F = f[f<f_cut]

In [0]:
def data_loader(select):
  
  global indexes
  global f_cut 


  this_trail = select[2]
  index = select[2]
  
  Indexes = indexes.copy()
  Indexes.pop(index)  # so, now Indexes is the array [0,1,2,...,17] with out the this_trial index

  samples = random.sample(Indexes, 11)
  samples += [this_trail]


  FFT_list = []

  for i in samples:
    result = np.fft.fft( subj_1['train_data_class{}'.format(select[1])][i].T )
    result = result.T
    result = result[f<f_cut, :]
    FFT_list.append(result)

  data =  np.mean(FFT_list, axis=0)
  label = select[1]
  return data, label

In [0]:
def data_loader_test_data(select):
  
  global test_data_indexes

  this_trail = select[2]
  index = select[2]-18
  
  Indexes = test_data_indexes.copy()
  Indexes.pop(index)  # so, now Indexes is the array [18,19,...,24] with out the this_trial index


  samples = random.sample(Indexes, 3)
  samples += [this_trail]




  FFT_list = []

  for i in samples:
    result = np.fft.fft( subj_1['train_data_class{}'.format(select[1])][i].T )
    result = result.T
    result = result[f<f_cut, :]
    FFT_list.append(result)


  data =  np.mean(FFT_list, axis=0)
  label = select[1]
  return data, label

In [0]:
def batch_loader(batch=[0,100]):
  # the batch=[a,b] can be 0...18000*4
  
  Label = []
  Data = []
  global selector
  

  for i in range(batch[0], batch[1]):
    data, label = data_loader(selector[int(i%len(selector))])
    Data.append(data)
    Label.append(label)

  Data = np.array(Data)
  Label = np.array(Label)

  return Data, Label

In [0]:
#data, label = batch_loader([0,2])
#data_, label_ = batch_loader_test_data([0,2])
#data_.shape

In [0]:
#plt.plot(F,np.abs(data[0][:,3]))
#plt.xlim([0,F[F==40]])

In [0]:
#plt.imshow(np.abs(data[0].T), aspect='auto', extent=[0,100,0,63])

In [0]:
def batch_loader_test_data(batch=[0,100]):
  # the batch=[a,b] can be 0...35*4
  
  Label = []
  Data = []
  global selector_test_data
  global selector
  

  for i in range(batch[0], batch[1]):
    data, label = data_loader_test_data(selector_test_data[int(i%len(selector_test_data))])
    '''
    L = len(selector[selector[:,1]==label])
    data_for_mean = []
    for j in range(6):
      data_, label_ = data_loader(selector[selector[:,1]==label][int((i+j)%L)])
      data_for_mean.append(data_)
      data = np.mean(data_for_mean, axis=0)
    '''
    Data.append(data)
    Label.append(label)

  Data = np.array(Data)
  Label = np.array(Label)

  return Data, Label

# The Model

In [0]:
from tensorflow import keras

In [0]:
model = keras.models.Sequential()

model.add(keras.layers.Conv1D(20, 1 , activation='relu', input_shape=(300,63)))
model.add(keras.layers.MaxPooling1D(2))
model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(40, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(30, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(10))
model.add(keras.layers.Dense(4,activation='softmax'))

In [0]:
model.compile(loss='sparse_categorical_crossentropy',
              metrics= ['accuracy'],
              optimizer='adam')

In [24]:
data, label = batch_loader([0, 50])
data.shape

(50, 300, 63)

In [28]:
model.fit(np.abs(data), label-1)

2/2 [==============================] - 0s 3ms/step - loss: 788.7399 - accuracy: 0.3000


In [29]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_2 (Conv1D)            (None, 300, 20)           1280      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 150, 20)           0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 3000)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 40)                120040    
_________________________________________________________________
dense_9 (Dense)              (None, 30)                1230      
_________________________________________________________________
dense_10 (Dense)             (None, 10)                310       
_________________________________________________________________
dense_11 (Dense)             (None, 4)                

In [23]:
data.shape

(50, 300, 63)

In [0]:

over_sample = 1
intervals = [[1000*i,1000*(i+1)] for i in range(10)]
for epoch in range(3):
  for interval in intervals:
    data, label = batch_loader(interval)
    data = np.abs(data)
    data_test, label_test=  batch_loader_test_data()
    data_test = np.abs(data_test)
    model.fit(data,  label-1, epochs=3, validation_data= (data_test, label_test-1))
    print('')
    print('############################')
    print('############################')
    print('interval:{}    and    epoch:{}'.format(interval, epoch))
    print('############################')
    print('############################')
    print('')
    

Epoch 1/3
32/32 [==============================] - 0s 11ms/step - loss: 69.8892 - accuracy: 0.8190 - val_loss: 543.0331 - val_accuracy: 0.2500
Epoch 2/3
32/32 [==============================] - 0s 5ms/step - loss: 0.3432 - accuracy: 0.9910 - val_loss: 580.4173 - val_accuracy: 0.2400
Epoch 3/3
32/32 [==============================] - 0s 5ms/step - loss: 0.1239 - accuracy: 0.9970 - val_loss: 622.2601 - val_accuracy: 0.2500

############################
############################
interval:[0, 1000]    and    epoch:0
############################
############################

Epoch 1/3
32/32 [==============================] - 0s 5ms/step - loss: 0.1294 - accuracy: 0.9970 - val_loss: 492.6938 - val_accuracy: 0.3500
Epoch 2/3
32/32 [==============================] - 0s 5ms/step - loss: 1.4306e-07 - accuracy: 1.0000 - val_loss: 481.9510 - val_accuracy: 0.3400
Epoch 3/3
32/32 [==============================] - 0s 5ms/step - loss: 6.0738e-07 - accuracy: 1.0000 - val_loss: 481.5867 - val_accura

In [60]:
data.shape

(50, 300, 63)

#Trash

In [42]:
model = keras.models.Sequential([
keras.layers.Conv2D(64, 7, activation="relu", padding="same",
input_shape=[28, 28, 1]),
keras.layers.MaxPooling2D(2),
keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
keras.layers.MaxPooling2D(2),
keras.layers.Conv2D(256, 3, activation="relu", padding="same"),
keras.layers.Conv2D(256, 3, activation="relu", padding="same"),
keras.layers.MaxPooling2D(2),
keras.layers.Flatten(),
keras.layers.Dense(128, activation="relu"),
keras.layers.Dropout(0.5),
keras.layers.Dense(64, activation="relu"),
keras.layers.Dropout(0.5),
keras.layers.Dense(10, activation="softmax")]


SyntaxError: ignored

In [0]:
model = keras.models.Sequential([
keras.layers.Conv2D(64, 7, activation="relu", padding="same",
input_shape=[28, 28, 1]),
keras.layers.MaxPooling2D(2),
keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
keras.layers.MaxPooling2D(2),
keras.layers.Conv2D(256, 3, activation="relu", padding="same"),
keras.layers.Conv2D(256, 3, activation="relu", padding="same"),
keras.layers.MaxPooling2D(2),
keras.layers.Flatten(),
keras.layers.Dense(128, activation="relu"),
keras.layers.Dropout(0.5),
keras.layers.Dense(64, activation="relu"),
keras.layers.Dropout(0.5),
keras.layers.Dense(10, activation="softmax")
                                 
])

In [45]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 28, 28, 64)        3200      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 14, 14, 128)       73856     
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 14, 14, 128)       147584    
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 7, 7, 128)         0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 7, 7, 256)         295168    
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 7, 7, 256)       